In [35]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

display_transform (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [43]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations : Obtaining rail points

In [12]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_mftip_frame_id = get_compiled_frameID(m, "rh_mftip")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
display(mftip_transform.origin)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.011
 -0.01
  0.44201000000000007

### Hand Visualization

In [100]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.011, -0.01, 0.34]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.011
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.34
 0.0  0.0  0.0   1.0

In [101]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_mftip")
display_frame(m, ls, "world")
display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [215]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))

Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [44]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the rail

In [13]:
# vm = Mechanism{Float64}("VirtualTracks")
# root_vm = root_frame(vm)

# FIRST TRACK : THE FOUR HAND FINGERS 
m = compile(robot)
kcache = new_kinematics_cache(m)  

# First point : rest position of the middle finger
rh_mftip_frame_id = get_compiled_frameID(m, "rh_mftip")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
display(mftip_transform.origin)

# Second point : 


3-element SVector{3, Float64} with indices SOneTo(3):
  0.011
 -0.01
  0.44201000000000007

### Addition of the multiples springs/dampers

Hand motion

In [91]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

m = compile(robot)
kcache = new_kinematics_cache(m)  

#Linking the four fingers to the same point, with non-zero equilibrium springs

#Definition of the constant target position
restpos = SVector(0.011, -0.01, 0.442)
add_coordinate!(vms, ConstCoord(restpos); id="Target position")

K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
D = SMatrix{3, 3}(10., 0., 0., 0., 10.0, 0., 0., 0., 10.)

#lightly constraining some joints to avoid unwanted motions
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_FFJ4_coord"); id = "ff j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_MFJ4_coord"); id = "mf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_RFJ4_coord"); id = "rf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ4_coord"); id = "lf j4 angular spring")
add_component!(vms, LinearSpring(100.0, ".robot.rh_WRJ1_coord"); id = "wr j1 angular spring")

#Linking fingers to this target 
rh_fftip_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_fftip"); id="ff position")
add_coordinate!(vms, ConstCoord(fftip_transform.origin - restpos); id="target ff vec")
add_coordinate!(vms, CoordSum("Target position", "target ff vec"); id="ff Target position")
#add_coordinate!(vms, FramePoin("cart_frame", fftip_transform.origin - restpos); id = "ff Target position) --> Later when there will be a cart 
add_coordinate!(vms, CoordDifference("ff Target position", "ff position"); id="ff position error")

add_component!(vms, LinearSpring(K, "ff position error");           id="ff position spring")
add_component!(vms, LinearDamper(D, "ff position error");           id="ff position damper")

rh_mftip_frame_id = get_compiled_frameID(m, "rh_mftip")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_mftip"); id="mf position")
add_coordinate!(vms, ConstCoord(mftip_transform.origin - restpos); id="target mf vec")
add_coordinate!(vms, CoordSum("Target position", "target mf vec"); id="mf Target position")
add_coordinate!(vms, CoordDifference("mf Target position", "mf position"); id="mf position error")

add_component!(vms, LinearSpring(K, "mf position error");           id="mf position spring")
add_component!(vms, LinearDamper(D, "mf position error");           id="mf position damper")

rh_rftip_frame_id = get_compiled_frameID(m, "rh_rftip")
rftip_transform = get_transform(kcache, rh_rftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_rftip"); id="rf position")
add_coordinate!(vms, ConstCoord(rftip_transform.origin - restpos); id="target rf vec")
add_coordinate!(vms, CoordSum("Target position", "target rf vec"); id="rf Target position")
add_coordinate!(vms, CoordDifference("rf Target position", "rf position"); id="rf position error")

add_component!(vms, LinearSpring(K, "rf position error");           id="rf position spring")
add_component!(vms, LinearDamper(D, "rf position error");           id="rf position damper")

rh_lftip_frame_id = get_compiled_frameID(m, "rh_lftip")
lftip_transform = get_transform(kcache, rh_lftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_lftip"); id="lf position")
add_coordinate!(vms, ConstCoord(lftip_transform.origin - restpos); id="target lf vec")
add_coordinate!(vms, CoordSum("Target position", "target lf vec"); id="lf Target position")
add_coordinate!(vms, CoordDifference("lf Target position", "lf position"); id="lf position error")

add_component!(vms, LinearSpring(K, "lf position error");           id="lf position spring")
add_component!(vms, LinearDamper(D, "lf position error");           id="lf position damper")


"lf position damper"

## Simulating the Robot

### Setting Up the Simulation

In [102]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())


function f_setup(cache) 
    return get_compiled_coordID(cache, "Target position")
end

function f_control(cache, t, args, extra)
    target_pos_id = args
    new_pos = SVector(0.011, -0.05, 0.32)
    new_pos2 = SVector(0.011, -0.0275, 0.275)
    new_pos3 = SVector(0.011, -0.01, 0.34)

    if t > 3
        cache[target_pos_id] = remake(cache[target_pos_id]; coord_data=ConstCoord(new_pos))
    end

    if t > 6
        cache[target_pos_id] = remake(cache[target_pos_id]; coord_data=ConstCoord(new_pos2))
    end

    if t > 9
        cache[target_pos_id] = remake(cache[target_pos_id]; coord_data=ConstCoord(new_pos3))
    end

end

tspan = (0., 12.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with attached fingers"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with attached fingers
ODE   0%|█                                              |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:01:27
ODE   4%|██                                             |  ETA: 0:01:07
ODE   6%|███                                            |  ETA: 0:00:58
ODE   8%|████                                           |  ETA: 0:00:52
ODE  11%|██████                                         |  ETA: 0:00:47
ODE  13%|███████                                        |  ETA: 0:00:44
ODE  15%|████████                                       |  ETA: 0:00:43
ODE  17%|█████████                                      |  ETA: 0:00:41
ODE  20%|██████████                                     |  ETA: 0:00:45
ODE  23%|███████████                                    |  ETA: 0:00:47
ODE  26%|█████████████                                  |  ETA: 0:00:52
ODE  29%|██████████████                                 |  ETA: 0:00:50
ODE  32%|█████

### Visualizing the Results

In [103]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)


savepath = joinpath(module_path, "docs/src/assets/shadowhand_sandbox.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);